In [ ]:
push!(LOAD_PATH,".")

In [ ]:
using Typeside, Chakra, PitchADT, MidiPitch, ChakraImp, MVS

In [ ]:
o1 = setAtt(delimit(),:pitch,NoteNumber(60))
o2 = setAtt(delimit(),:pitch,NoteNumber(62))
o3 = setAtt(delimit(),:pitch,NoteNumber(68))
o4 = setAtt(delimit(),:pitch,NoteNumber(60))
o5 = setAtt(delimit(),:pitch,NoteNumber(62))
o6 = setAtt(delimit(),:pitch,NoteNumber(70))

In [ ]:
s = [o1,o2,o3,o4,o5,o6]

In [ ]:
v = AtomicViewpoint(Pitch,:pitch)
vp_type(v)
vp_apply(v,s)

In [ ]:
v2 = LinkedViewpoint(v,DelayedViewpoint(v,1))
#vp_type(v2)
vp_apply(v2,s)

In [ ]:
v3 = DerivedViewpoint(PitchInterval,v2,PitchADT.diff)
vp_type(v3)
vp_apply(v3,s)

In [ ]:
v4 = DerivedViewpoint(Pitch,v3,x->PitchADT.shift(x,NoteNumber(40)))
vp_type(v4)
vp_apply(v4,s)

In [ ]:
seq_delay(s,1)

In [ ]:
MVS.vp_map(v3,s)